In [212]:
import pandas as pd

In [226]:
df = pd.read_csv("old.csv")

In [227]:
df.head()

,OG Sequence,OG PPL,Unguided Sequence,Unguided PPL,Unguided Solubility,Optimized Sequence,Optimized PPL,Optimized Solubility
0,LSRLMIAGLMVFLVLSLVVLLAGRLpFtpqpapvtgntyrtyvnda...,2.2562,LPPKIIKDPMINAAAVLIRDTGRAMPQTPQPAPVTGNTYRTYVNDA...,2.3631,0.6463,QSPQLGRVPMMLYVEHDILTVSGAAPLTPQPAPVTGNTYRTYVNDA...,2.4488,0.8659
1,ssriagdttgkytpnifspetpmdrsfshvpknpfweawvfrrdni...,1.7978,SSRIAGDTTGKYTPNIFSPETPMDRSFSHVPKNPFWEAWVFRRDNI...,1.6880,0.7788,NaN,0.0000,0.0000
2,sllknyppsylypfrhpkPEGVIEKVLFNLGSLFRSAGQGMDELGs...,1.2912,SLLKNYPPSYLYPFRHPKFKIALERYFGAGFKFLGVKLNGLKDHGS...,1.2883,1.0000,NaN,0.0000,0.0000
3,amktlekVNYKGFIWPLAVGIVLWLITPWrpgglsvqAWEMFAIFV...,6.6295,AMKTLEKLVSCSREFVKTFKVLFDEAKAWRPGGLSVQEIENITKEN...,7.0583,0.6517,AMKTLEKFFLKNDVSKFALERVILDSRFDRPGGLSVQIDFICFCGV...,6.8439,0.7128
4,amktlekVNYKGFIWPLAVGIVLWLITPWrpgglsvqAWEMFAIFV...,6.6968,AMKTLEKHLKLEDPKKVSIGPVIFFFQFVRPGGLSVQGENRIMADG...,6.8673,0.6619,AMKTLEKTTLKVSKQYLGLDAPQDNDFPGRPGGLSVQKPFETGWRT...,7.1743,0.7882


In [228]:
df = df[df["Optimized Solubility"] != 0]

In [229]:
import numpy as np
import torch
from tqdm import trange

In [249]:
# @torch.no_grad()
def compute_pseudo_perplexity(model, tokenizer, protein_seq): 
    '''
    Computes the pseudo-perplexity of a protein sequence using the ESM-2-650M pLM.
    '''
    tensor_input = tokenizer(protein_seq,
    return_tensors='pt').to(model.device)["input_ids"]
    # print(tensor_input.size(), tensor_input)

    total_loss = 0
    
    # print(protein_seq)

    # Loop through each token in the sequence
    for i in trange(-len(protein_seq)-1, -1):

        # Create a copy of the original tensor
        masked_input = tensor_input.clone()
        # print(masked_input.size())
        # Mask one token at a time
        masked_input[:, i] = tokenizer.mask_token_id

        # Create labels
        labels = torch.full(tensor_input.shape,-100).to(model.device)
        labels[:, i] = tensor_input[:,i]

        # Get model prediction and loss
        with torch.no_grad():
            outputs = model(masked_input, labels=labels)
            total_loss += outputs.loss.item()
            # print(outputs.loss)

        # Calculate the average loss
        avg_loss = total_loss / len(protein_seq)
        # break

    # Calculate pseudo perplexity
    # print(avg_loss)
    pseudo_perplexity = np.exp(avg_loss)
    # print(pseudo_perplexity)
    return pseudo_perplexity


In [257]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t33_650M_UR50D").cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [251]:
df = df.reset_index()

ValueError: cannot insert level_0, already exists

In [ ]:
for i in trange(69, len(df)):

    og_seq = df.loc[i]["OG Sequence"]
    ug_seq = df.loc[i]["Unguided Sequence"]
    g_seq = df.loc[i]["Optimized Sequence"]
    ppl1 = compute_pseudo_perplexity(model, tokenizer, og_seq.upper())
    ppl2 = compute_pseudo_perplexity(model, tokenizer, ug_seq.upper())
    ppl3 = compute_pseudo_perplexity(model, tokenizer, g_seq.upper())
    
    df.loc[i, "OG PPL"] = ppl1
    df.loc[i, "Unguided PPL"] = ppl2
    df.loc[i, "Optimized PPL"] = ppl3

100%|██████████| 781/781 [00:26<00:00, 29.92it/s]


2.3826680925008294
10.83376983698722


100%|██████████| 781/781 [00:26<00:00, 29.78it/s]


2.8635897996636266
17.52432296524871


  0%|          | 1/282 [01:18<6:07:59, 78.58s/it]

...

0.9390450326938281
2.557537886702468


 33%|███▎      | 94/282 [41:59<3:26:32, 65.92s/it]

1.3319087699953533
3.7882674220548984


100%|██████████| 825/825 [00:28<00:00, 28.78it/s]


0.5984102158478741
1.8192243262025816


In [182]:
df.loc[1, "OG Sequence"]

'amktlekVNYKGFIWPLAVGIVLWLITPWrpgglsvqAWEMFAIFVATIVGCITKPLPIGGTTLLGMVVTVLVGLAPVKdvvnskgvviqTGILSSFGNSAAWLIAMAFIMAHGIsktglgnrvayvmiekfgkrsigiGYAITGLELMMGALIPSNSARTGGVTWPVVEsisksydskpndpsrkkigAYLDFMAFHANILSTALFITGAAPNLvaqqmaaqkgyqmSWVSWFWAALVPVLVATVIIPLVIYkMYppevketpnaknwaddklkemgpiskpeKIMATVFCLAILLWVLSGFFKIPQLDSAFVAFLAVTLLLITGVLSMEDALHETGAWNILIWLSILIFMAGKLisygFiawFakfiqsevhginwGLvLVVLILLMFYTHYFFASGTAHMtalylpfltvatamgaplgLsaMLLAFTGVINASTTHYANGPASILATTGYVKQSEWWKMNFILGLIYMVIFGIVGTIWMKIIGIW\n'

In [222]:
df.to_csv("processed.csv")